<a href="https://colab.research.google.com/github/vrc-lt/SlideConverters/blob/main/colab/unaslides_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UnaSlides/VRC-LT スライド変換ツール
pdf形式スライドファイルを、画面アスペクト比16:9のmp4動画ファイルに変換します。

出力する解像度を選択してから、ランタイム→すべてのセルを実行(Ctrl+F9)で処理を開始し、変換したいpdfファイルをアップロードしてください。処理が終了したら、変換後のmp4ファイルをダウンロードできます。

- UnaSlides: https://booth.pm/ja/items/4141632
- VRC-LT: https://vrc-lt.org/about/

In [ ]:
#@title 動画タイプ・出力解像度を選択してからランタイムを実行してください (UnaSlides: 1ページあたり1秒, VRC-LT: 1ページあたり2秒)
output_framerate_type = "UnaSlides" #@param ["UnaSlides", "VRC-LT"]
output_framerate = {"VRC-LT": ("1/2", "vrclt"), "UnaSlides": ("1", "unaslides")}[output_framerate_type]
output_resolution = "720p" #@param ["720p", "1080p", "1440p", "2160p"]
output_resolution_xy = {"720p": (1280, 720), "1080p": (1920, 1080), "1440p": (2560, 1440), "2160p": (3840, 2160)}[output_resolution]

In [ ]:
#@title pdfファイルをアップロードしてください
from google.colab import files
import os
!rm -f *.pdf
result = files.upload()
pdf_file_name = list(result.keys())[0]
base_file_name = os.path.splitext(pdf_file_name)[0]

In [ ]:
#@title 環境をセットアップしています...
!sudo apt update
!sudo apt install --no-install-recommends ffmpeg poppler-utils poppler-data fonts-liberation2 fonts-noto-cjk
!mkdir -p ~/.config/fontconfig/
!echo -e "<?xml version='1.0'?>\n<!DOCTYPE fontconfig SYSTEM 'fonts.dtd'>\n<fontconfig>\n<alias><family>serif</family><prefer><family>Noto Serif</family><family>Noto Serif CJK JP</family></prefer></alias>\n<alias><family>sans-serif</family><prefer><family>Noto Sans</family><family>Noto Sans CJK JP</family></prefer></alias>\n<alias><family>monospace</family><prefer><family>Noto Sans Mono</family><family>Noto Sans Mono CJK JP</family></prefer></alias>\n<alias><family>Ryumin</family><prefer><family>Noto Serif</family><family>Noto Serif CJK JP</family></prefer></alias>\n<alias><family>GothicBBB</family><prefer><family>Noto Sans</family><family>Noto Sans CJK JP</family></prefer></alias>\n</fontconfig>" > ~/.config/fontconfig/fonts.conf
!fc-cache -f

In [ ]:
#@title 変換しています...
output_resolution_x = output_resolution_xy[0]
output_resolution_y = output_resolution_xy[1]
output_framerate_fps = output_framerate[0]
output_framerate_ext = output_framerate[1]
mp4_file_name = base_file_name + "." + str(output_resolution_y) + "p." + output_framerate_ext + ".mp4"
!rm -f image-*.png *.mp4
!echo -e "\e[36;1m[pdftoppm]\e[0m $pdf_file_name -> image-*.png $output_resolution_xy"
!pdftoppm -scale-to-x -1 -scale-to-y $output_resolution_y -png "$pdf_file_name" image
!echo -e "\e[36;1m[ffmpeg]\e[0m image-*.png -> $mp4_file_name"
!ffmpeg -y -pattern_type glob -r $output_framerate_fps -i "image-*.png" -vf "crop=min(ih*16/9\,iw):ih,scale=-2:$output_resolution_y:flags=lanczos,pad=x=-2:aspect=16/9" -c:v libx264 -r 30 -pix_fmt yuv420p "$mp4_file_name"

In [ ]:
#@title mp4ファイルをダウンロードします...
files.download(mp4_file_name)